# Intro

This notebooks takes a pre-processed dataframe, and encodes the data so it can be used to train Banksformer.  


The input dataframe requires following columns: 
- tcode - String, encodes transaction type 
- amount - float, transcation amount (not log) 
- account_id - int, associates transactions with account
- age - int, clients age
- datetime - datetime object, date of transaction
- day, month, dow - all ints, encode day, month and day of week
- td - int/float, time delta, encodes number of days since the last transaction 

The encoded data will be tensor of shape (n_samples, max_seq_len, feats_per_step).

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import time

import pickle
from datetime import date
import tensorflow as tf

## Setup

In [2]:
ds_suffix = "-uk"
max_seq_len = 20
min_seq_len = 5

### Load dataframe

In [3]:
df = pd.read_csv(f"stored_data/final_df-{ds_suffix}.csv")
df

,unnamed: 0,index,account_id,amount,balance,date,description,flag,id,type,age,datetime,month,day,dow,year,dtme,tcode,td
0,28979,0,0014d0ef29aa9f93,-45.66,228.34,2017-04-24 11:00:00.000000,Energy,Utility Bill,38a21d894127b49d,Debit,-1,2017-04-24 11:00:00.000000,4,24,0,2017,6,Energy__Utility Bill__Debit,0.0
1,30006,1,0014d0ef29aa9f93,2842.27,3070.61,2017-04-24 17:36:55.000000,Monthly,Income,3941e50e4613a49a,Credit,-1,2017-04-24 17:36:55.000000,4,24,0,2017,6,Monthly__Income__Credit,0.0
2,32220,2,0014d0ef29aa9f93,-167.25,2903.36,2017-04-25 16:00:00.000000,Credit Card Payment,Credit Card,c09e8105dcfc85b7,Debit,-1,2017-04-25 16:00:00.000000,4,25,1,2017,5,Credit Card Payment__Credit Card__Debit,0.0
3,35526,3,0014d0ef29aa9f93,-40.58,2862.78,2017-04-27 16:00:00.000000,Water Bill,Utility Bill,81e4ffc8df86920e,Debit,-1,2017-04-27 16:00:00.000000,4,27,3,2017,3,Water Bill__Utility Bill__Debit,2.0
4,94259,4,0014d0ef29aa9f93,-43.16,2819.62,2017-05-23 17:00:00.000000,Energy,Utility Bill,c0ddd8f8a923d40f,Debit,-1,2017-05-23 17:00:00.000000,5,23,1,2017,8,Energy__Utility Bill__Debit,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,89757,23,ffd5620bcbf69023,-34.82,2107.16,2017-05-20 15:50:57.000000,Shopping - Household Goods,Card Transaction,908ac96c72f6c3b7,Debit,-1,2017-05-20 15:50:57.000000,5,20,5,2017,11,Shopping - Household Goods__Card Transaction__...,0.0
99996,91385,24,ffd5620bcbf69023,-36.50,2070.66,2017-05-22 08:00:00.000000,Water Bill,Utility Bill,66d16692309a0738,Debit,-1,2017-05-22 08:00:00.000000,5,22,0,2017,9,Water Bill__Utility Bill__Debit,1.0
99997,91449,25,ffd5620bcbf69023,-38.13,2032.53,2017-05-22 09:00:00,Energy,Utility Bill,b71e06b7164af1e3,Debit,-1,2017-05-22 09:00:00,5,22,0,2017,9,Energy__Utility Bill__Debit,0.0
99998,93444,26,ffd5620bcbf69023,-42.59,1989.94,2017-05-22 20:47:15,Bar/Pub,Card Transaction,8a28f7eeff2dbabb,Debit,-1,2017-05-22 20:47:15,5,22,0,2017,9,Bar/Pub__Card Transaction__Debit,0.0


## Ensure correct folders exist

In [4]:
folders = ['generated_data',
 'stored_data',
 'checkpoints',
 'generation_results',
 'data',
 'my_lib']


for f in folders:
    if not os.path.exists(f):
        os.mkdir(f)

## Encode

In [5]:
df['amount'] = np.abs(df.amount) # model only magnitude of amount, use categorical feature to distinguish credit/debit

In [6]:
from my_lib.encoding import preprocess_df, bulk_encode_time_value
from field_config import CAT_FIELDS

preprocess_df(df, CAT_FIELDS, ds_suffix)

DATA_KEY_ORDER is ['tcode_num', 'dow', 'month', 'day', 'dtme', 'td_sc', 'log_amount_sc']
LOSS_TYPES are: day - scce, dtme - scce, dow - scce, month - scce, td_sc - pdf, log_amount_sc - pdf, tcode_num - scce
If this is not correct, edit field_config.py and re-run notebook
iniy!
Wrote encoding info to stored_data/DataEncoder--uk.pickle


In [7]:
# from my_lib.field_config import *

from field_config import get_field_info, DATA_KEY_ORDER, CLOCK_DIMS, INP_ENCODINGS, TAR_ENCODINGS


def count_seqs_in_df(df):
    gb_aid = df.groupby("account_id")["account_id"]

    full_seqs_per_acct = gb_aid.count() // max_seq_len

    n_full_seqs = sum(full_seqs_per_acct)
    n_part_seqs = sum(gb_aid.count() - full_seqs_per_acct*max_seq_len >= min_seq_len)
    
    return n_full_seqs + n_part_seqs






def seq_to_inp_tensor(seq, inp_tensor, seq_i, seq_len):

    for k in DATA_KEY_ORDER:

        depth = FIELD_DIMS_IN[k]
        st = FIELD_STARTS_IN[k]
        enc_type = INP_ENCODINGS[k]
        
        if enc_type == "oh":
            x = tf.one_hot(seq[k], depth).numpy()
        elif enc_type == "cl":
            max_val = CLOCK_DIMS[k]
            x = bulk_encode_time_value(seq[k], max_val)
        elif enc_type == "raw":
            x = np.expand_dims(seq[k], 1)
        else:
            raise Exception(f"Got invalid enc_type: {enc_type}")
            
            

        inp_tensor[seq_i,:seq_len, st:st+depth] = x




def seq_to_targ_tensor(seq, tar_tensor, seq_i, seq_len):

    for k in DATA_KEY_ORDER:
        
        depth = FIELD_DIMS_TAR[k]
        st = FIELD_STARTS_TAR[k]
        enc_type = TAR_ENCODINGS[k]
        
        if enc_type == "cl-i":
            max_val = CLOCK_DIMS[k]
            x = np.expand_dims(seq[k]%max_val, 1)
        elif enc_type == "raw":
            x = np.expand_dims(seq[k], 1)
        else:
            raise Exception(f"Got invalid enc_type: {enc_type}")
            
        
        tar_tensor[seq_i,:seq_len, st:st+depth] = x

        
FIELD_DIMS_IN, FIELD_STARTS_IN, FIELD_DIMS_TAR, FIELD_STARTS_TAR, FIELD_DIMS_NET, FIELD_STARTS_NET = get_field_info(ds_suffix)

In [8]:
n_seqs = count_seqs_in_df(df)
n_steps = max_seq_len
n_feat_inp = sum(FIELD_DIMS_IN.values())
n_feat_tar = sum(FIELD_DIMS_TAR.values())

inp_tensor = np.zeros((n_seqs, n_steps, n_feat_inp))
tar_tensor = np.zeros((n_seqs, n_steps, n_feat_tar))

inp_tensor.shape, tar_tensor.shape

((6983, 20, 54), (6983, 20, 7))

In [9]:
seq_i = 0
rows_per_acct = {}
alert_every = 2000
attribute = "age_sc"


attributes = np.zeros(n_seqs)
start_time = time.time()
for acct_id, group in df.groupby("account_id"):
    rows_per_acct[acct_id] = []
    
    for i in range(len(group) // max_seq_len + 1):

        n_trs = len(group)
        start = i*max_seq_len
        seq_len = min(max_seq_len, n_trs - start)   

        if seq_len >= min_seq_len:
            seq_to_inp_tensor(group.iloc[start:start+seq_len], inp_tensor, seq_i, seq_len)
            seq_to_targ_tensor(group.iloc[start:start+seq_len],tar_tensor, seq_i, seq_len)
#             tar_tensor[seq_i,:seq_len,:] = seq_to_targ_tensor(group.iloc[start:start+seq_len])
            attributes[seq_i] = group["age"].iloc[0]

            rows_per_acct[acct_id].append(seq_i)
            seq_i += 1
            
            if seq_i % alert_every == 0:
                print(f"Finished encoding {seq_i} of {n_seqs} seqs")
                
 
# Add conditioning info (attribute) to first timestep of inp
inp_tensor = np.concatenate([np.repeat(attributes[:, None, None], n_feat_inp, axis=2), 
                             inp_tensor], 
                             axis=1)
print(f"Took {time.time() - start_time:.2f} secs")

2022-06-29 11:51:54.800422: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Finished encoding 2000 of 6983 seqs
Finished encoding 4000 of 6983 seqs
Finished encoding 6000 of 6983 seqs
Took 110.76 secs


In [10]:
inp_tensor.shape, tar_tensor.shape, attributes.shape

((6983, 21, 54), (6983, 20, 7), (6983,))

## Save

In [11]:
np.save(f"stored_data/inp_tensor-{ds_suffix}", inp_tensor)
np.save(f"stored_data/tar_tensor-{ds_suffix}", tar_tensor)
np.save(f"stored_data/attributes-{ds_suffix}", attributes)

In [12]:
with open(f"stored_data/rows_per_acct-{ds_suffix}.pickle", "wb") as f:
    pickle.dump(rows_per_acct, f) 